# Sistemas de Recomendação

Essa prática irá apresentar passo a passo a implementação de um sistema de recomendação baseado em filtros colaborativos, onde será gerada a matriz de ratings, e serão calculadas as similaridades entre os usuários.

Para iniciar vamos importar a biblioteca pandas, do Python, que possui estruturas de dados muito úteis para manipular as informações.

Teremos duas bases de dados básicas: 

- Dados de ratings so sistema MovieLenz, que possui ratings observados para filmes. O arquivo referente a esses dados é o ratings.csv.

- Dados dos filmes, para que possamos obter os nomes dos filmes recomendados e ter uma noção do que foi recomendado com o nosso sistema de recomendação.

As próximas célular do nosso notebook fazem a leitura desses arquivos e imprimem o ínicio deles após serem estruturados em um pandas dataframe. Execute essas quatro células e veja como os dados estão estruturados.

Para executar uma célula do nosso notebook, basta utilizar o atalho shift+enter.

In [1]:
import pandas as pd
dataFile='ratings.csv'
data=pd.read_csv(dataFile,sep=",",header=0,names=["userId","movieId","rating","timestamp"])

In [3]:
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
movieFile='movies.csv'
movies=pd.read_csv(movieFile,sep=",",header=0,names=["movieId","title","genres"])

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Agora que temos os dados carregados no formato visto acima, vamos criar um método bem simples, que irá receber como parâmetro o ID de um filme e irá retornar o seu título e os gêneros associados a ele.

Nas duas células em branco faça a mesma chamada feita para o filme 2, na célula abaixo, porém para o filme 5 e o 6, apenas para critério de teste.

In [6]:
#definição de um método para obter o título e os gêneros associados e um filme.
#veja que o nosso dataframe "movies" definido acima, é utilizado para a consulta, baseado no ID do filme
def movieMeta(movieId):
    title = movies.at[movieId,"title"]
    genre = movies.at[movieId,"genres"]
    return title, genre

#Exemplo de chamada do método que retorna os dados do filmes
movieMeta(2)

('Grumpier Old Men (1995)', 'Comedy|Romance')

In [6]:
#Faça aqui a chamada do método "movieMeta" passando o filme 5 como parâmetro


In [7]:
movieMeta(5)

('Heat (1995)', 'Action|Crime|Thriller')

In [8]:
#Faça aqui a chamada do método "movieMeta" passando o filme 9 como parâmetro


In [8]:
movieMeta(9)

('GoldenEye (1995)', 'Action|Adventure|Thriller')

In [9]:
#código que obtem os ids dos filmes existentes no nosso dataframe movies.
data = data[data["movieId"].isin(movies.index)]

Agora que temos uma função que retorna os dados básicos do filme, para vermos o que foi recomendado, precisamos criar uma para retornar os filmes favoritos de um usuário.

Para isso:
- Recuperamos a lista de filmes de um usuário.
- Ordenamos de forma decrescente com base nos ratings do usuário.
- Pegamos os N primeiros itens dessa lista ordenada.

Toda essa implementação esta presente no método "faveMovies" abaixo. Ele recebe como parâmetro o ID do usuário e retorna os N itens mais relevantes para ele, com base no rating observado. Esse método será utilizado mais adiante para encontrarmos os vizinhos mais próximos desse usuário, com base no ratings similares aos seus ratings observados.

In [10]:
#método que retorna os N filmes com maior rating para um determinado usuário.
def faveMovies(user,N):
    #userRatings recebe a lista de ratings do usuário passado como parâmetro
    userRatings = data[data["userId"]==user]
    #SortedRatings recebe os N primeiros itens dessa lista ordenada de forma decrescente
    sortedRatings = pd.DataFrame.sort_values(userRatings,['rating'],ascending=[0])[:N]
    #o método movieMeta é aplicado nos N itens mais relevantes, para que a coluna title
    #seja criada no nosso dataframe sortedRatings, para que possamos visualizar.
    sortedRatings["title"] = sortedRatings["movieId"].apply(movieMeta)
    return sortedRatings

Uma vez definida a função que pega os N filmes mais relevantes, vamos fazer um teste com os 10 filmes mais relevantes para o usuário 1. Faça a chamada desse método nas duas células vazias abaixo, para o usuário 2 e o 3, respectivamente.

In [58]:
faveMovies(1,60)

,userId,movieId,rating,timestamp,title
4,1,1172,4.0,1260759205,"(Gridlock'd (1997), Crime)"
13,1,2105,4.0,1260759139,"(Besieged (a.k.a. L' Assedio) (1998), Drama)"
12,1,1953,4.0,1260759191,"(At First Sight (1999), Drama)"
8,1,1339,3.5,1260759125,"(Red Corner (1997), Crime|Thriller)"
19,1,3671,3.0,1260759117,"(War of the Roses, The (1989), Comedy|Drama)"
1,1,1029,3.0,1260759179,"(Highlander (1986), Action|Adventure|Fantasy)"
2,1,1061,3.0,1260759182,"(Hype! (1996), Documentary)"
14,1,2150,3.0,1260759194,"(General's Daughter, The (1999), Crime|Drama|M..."
17,1,2455,2.5,1260759113,"(Pokémon: The First Movie (1998), Adventure|An..."
0,1,31,2.5,1260759144,"(Twelve Monkeys (a.k.a. 12 Monkeys) (1995), My..."


In [12]:
#Faça a chamada do método faveMovies para o usuário 2, com 15 filmes como parâmetro.

In [13]:
faveMovies(2,15)

,userId,movieId,rating,timestamp,title
83,2,551,5.0,835355767,"(Carried Away (1996), Drama|Romance)"
22,2,39,5.0,835355604,"(Richard III (1995), Drama|War)"
45,2,266,5.0,835355586,"(Pulp Fiction (1994), Comedy|Crime|Drama|Thril..."
91,2,592,5.0,835355395,"(Delta of Venus (1995), Drama)"
90,2,590,5.0,835355395,"(Butterfly Kiss (1995), Drama|Thriller)"
89,2,589,5.0,835355697,"(True Crime (1996), Mystery|Thriller)"
85,2,585,5.0,835355817,"(Promise, The (Versprechen, Das) (1995), Drama..."
37,2,222,5.0,835355840,"(Heavyweights (Heavy Weights) (1995), Children..."
29,2,150,5.0,835355395,"(Jeffrey (1995), Comedy|Drama)"
44,2,265,5.0,835355697,"(Pyromaniac's Love Story, A (1995), Comedy|Rom..."


In [15]:
#Faça a chamada do método faveMovies para o usuário 3, com 8 filmes como parâmetro.

In [14]:
faveMovies(3,8)

,userId,movieId,rating,timestamp,title
133,3,3949,5.0,1298863174,"(Sandlot, The (1993), Children|Comedy|Drama)"
113,3,1197,5.0,1298932770,"(Crash (1996), Drama|Thriller)"
130,3,2959,5.0,1298862874,"(Bird on a Wire (1990), Action|Comedy|Romance)"
101,3,318,5.0,1298862121,"(Crow, The (1994), Action|Crime|Fantasy|Thriller)"
103,3,356,5.0,1298862167,"(Mirage (1995), Action|Thriller)"
119,3,1721,4.5,1298923236,"(Return to Paradise (1998), Crime|Drama|Romanc..."
100,3,296,4.5,1298862418,"(Tom & Viv (1994), Drama)"
111,3,778,4.0,1298863157,"(Angel on My Shoulder (1946), Crime|Drama)"


In [15]:
#verificar as dimensões do nosso dataframe contendo os ratings.
#ao executar essa célula, veja que temos cerca de 80000 linhas e 4 colunas (userId, movieId, rating, timestamp)
data.shape

(84739, 4)

In [16]:
#userPerMOVIE recebe quantos usuários viram cada filme.
usersPerMOVIE = data.movieId.value_counts()
usersPerMOVIE.head(10)

356     341
296     324
318     311
593     304
260     291
480     274
2571    259
1       247
527     244
589     237
Name: movieId, dtype: int64

In [17]:
#ao verificar a dimensão do resultado de usersPerMOVIE, vemos que cerca de 5642 filmes foram vistos por
#pelo menos um usuário.
usersPerMOVIE.shape

(5642,)

In [18]:
#fazemos o mesmo para verificar quantos usuários viram um certo filme
MOVIESsPerUser = data.userId.value_counts()
MOVIESsPerUser.head()

564    1868
547    1627
452    1308
15     1166
468    1022
Name: userId, dtype: int64

In [19]:
MOVIESsPerUser.shape

(671,)

Agora vamos obter com a célula abaixo os filmes vistos por mais de 10 usuários e na próxima os usuários que viram mais de 10 filmes, como uma forma de redução simples de dimensionalidade.

In [20]:
data = data[data["movieId"].isin(usersPerMOVIE[usersPerMOVIE>10].index)]

In [21]:
data = data[data["userId"].isin(MOVIESsPerUser[MOVIESsPerUser>10].index)]

Feito isso, vamos utilizar o método pivot_table do pandas dataframe, para gerar nossa matriz de ratings, onde os valores serão os ratings, os índices, ou linhas, serão os usuários e as colunas o ID dos filmes, com base nos nossos dados de ratings, lidos do arquivo rating.csv no começo do notebook.

Percebam que essa matriz possui diversos valores NaN, que seriam os nulos, onde não existem ratings observados, conforme visto nas aulas teóricas.

In [22]:
userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['userId'], columns=['movieId'])

In [23]:
userItemRatingMatrix.head()

movieId,1,2,3,4,5,6,7,9,10,11,...,8957,8958,8961,8966,8970,8972,8981,8983,8984,8985
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
userItemRatingMatrix.shape

(666, 1759)

Agora vamos definir dois usuários para obter os ratings observados deles, conforme as 3 células abaixo.

Percebam que o usuário 1 utilizado como exemplo para exibir os 100 primeiros ratings, possui diversos valores nulos, e apenas alguns com ratings observados.

In [30]:
user1 = 1
user2 = 20

In [31]:
user1Ratings = userItemRatingMatrix.transpose()[user1]
user1Ratings.head(100)

movieId
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
9      NaN
10     NaN
11     NaN
12     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
28     NaN
29     NaN
31     2.5
32     NaN
34     NaN
35     NaN
36     NaN
      ... 
125    NaN
135    NaN
140    NaN
141    NaN
144    NaN
145    NaN
147    NaN
150    NaN
151    NaN
153    NaN
154    NaN
156    NaN
158    NaN
160    NaN
161    NaN
162    NaN
163    NaN
164    NaN
165    NaN
168    NaN
170    NaN
172    NaN
173    NaN
175    NaN
176    NaN
177    NaN
180    NaN
181    NaN
185    NaN
186    NaN
Name: 1, Length: 100, dtype: float64

In [32]:
user2Ratings = userItemRatingMatrix.transpose()[user2]

Agora precisamos definir uma forma de similaridade entre os usuários ou produtos. Iremos trabalhar apenas com os usuários.

Para isso foi utilizada a distância de hamming, existente na biblioteca scipy.

Poderia ser utilizado também a correlação de Perason ou de cossenos, existentes nessa biblioteca, porém foi selecionada a distância de hamming para apresentar outra forma de cálculo de similaridade, baseada na distância entre os usuários.

No caso abaixo ela é calculada entre o usuario1 e usuario2, definidos anteriormente.

Feito isso vamos criar o método "distance" que irá fazer esse cálculo para quaiquer dois usuários passados como parâmetro.

In [33]:
from scipy.spatial.distance import hamming 
hamming(user1Ratings,user2Ratings)

1.0

In [34]:
#método para calcular a métrica de correlação, baseada em distância, entre dois usuários.
#os parâmetros de entrada são os dois usuários.
import numpy as np
def distance(user1,user2):
        try:
            #recupera os ratings dos dois usuários e depois é aplicado o método hamming, para calcular
            #a distância, que será retornada.
            user1Ratings = userItemRatingMatrix.transpose()[user1]
            user2Ratings = userItemRatingMatrix.transpose()[user2]
            distance = hamming(user1Ratings,user2Ratings)
        except: 
            distance = np.NaN
        return distance 

Veja abaixo o exemplo para calcular a distância entre o usuário 6 e o 5, e faça o mesmo para o usuário 2 em relação ao 3 e para o usuário 1 em relação ao 2.

In [35]:
distance(6,5)

0.9994314951677089

In [33]:
#faça o cálculo da similaridade entre 2 e 3

In [36]:
distance(2,3)

0.9982944855031268

In [35]:
#faça o cálculo da similaridade entre 6 e 0

In [37]:
distance(6,1)

1.0

In [38]:
#agora iremos filtrar o usuário 6 e calcular a distância entre ele e todos os demais usuários
#perceba que foram selecionados todos os ratings de todos os usuários diferentes de 6
user = 6
allUsers = pd.DataFrame(userItemRatingMatrix.index)
allUsers = allUsers[allUsers.userId!=user]
allUsers.head(10)

,userId
0,1
1,2
2,3
3,4
4,5
6,7
7,8
8,9
9,10
10,11


In [39]:
#as distâncias são então calculadas aplicando a função de distância para todos os usuários em relação ao
#usuário 6
#Isso é feito com a expressão lambda, que ira pegar a lista de todos os userId do nosso dataframe allusers
#e aplicar a função distance em relação ao nosso usuário 6.
#tudo isso com uma simples linha de código.
allUsers["distance"] = allUsers["userId"].apply(lambda x: distance(user,x))

In [40]:
allUsers.head()

,userId,distance
0,1,1.000000
1,2,1.000000
2,3,1.000000
3,4,1.000000
4,5,0.999431


Agora podemos selecionar os K vizinhos mais próximos ordenando esse resultado das distâncias e pegando os K primeiros usuários resultantes dessa ordenação.

In [41]:
K = 10
KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["userId"][:K]

In [42]:
KnearestUsers

72      73
465    468
252    253
118    119
29      30
104    105
649    654
544    547
558    561
200    201
Name: userId, dtype: int64

Agora vamos criar um método que aplica essa seleção dos usuários diferentes do que desejamos fazer as recomendações, cálcula as similaridades e ordena o resultado, retornando apenas os mais próximos.

Esse método é o apresentado abaixo, chamado de nearestNeighbors, que recebe como parâmetro o usuário e o número de vizinhos.

In [43]:
def nearestNeighbors(user,K=10):
    allUsers = pd.DataFrame(userItemRatingMatrix.index)
    allUsers = allUsers[allUsers.userId!=user]
    allUsers["distance"] = allUsers["userId"].apply(lambda x: distance(user,x))
    KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["userId"][:K]
    return KnearestUsers

In [44]:
KnearestUsers = nearestNeighbors(user)

In [45]:
KnearestUsers

72      73
465    468
252    253
118    119
29      30
104    105
649    654
544    547
558    561
200    201
Name: userId, dtype: int64

In [46]:
#vamos então com o código abaixo, filtrar a matriz de ratings, apenas com os usuários correspondentes
#aos vizinhos mais próximos.
NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
NNRatings

movieId,1,2,3,4,5,6,7,9,10,11,...,8957,8958,8961,8966,8970,8972,8981,8983,8984,8985
userId,,,,,,,,,,,,,,,,,,,,,
30,4.0,2.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,5.0,2.5,NaN,NaN,NaN,4.5,NaN,NaN,3.0,NaN,...,4.0,NaN,3.0,NaN,NaN,3.0,NaN,NaN,2.5,NaN
105,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.5,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,2.0,3.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,5.0,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN
253,4.5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
468,4.0,2.0,2.5,NaN,NaN,3.5,NaN,NaN,3.0,2.0,...,NaN,3.0,3.5,NaN,3.0,NaN,3.0,NaN,NaN,NaN
547,3.5,NaN,NaN,NaN,NaN,2.5,2.0,NaN,NaN,3.0,...,NaN,4.5,NaN,4.0,3.5,NaN,4.0,NaN,2.5,NaN
561,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,...,NaN,NaN,4.5,NaN,NaN,4.0,NaN,NaN,4.0,NaN


Cálculamos então e média dos ratings dos vizinhos mais próximos, para determinar os ratings não observados para o usuário que desejamos.

In [47]:
avgRating = NNRatings.apply(np.nanmean).dropna()
avgRating.head()

C:\Anaconda3\lib\site-packages\pandas\core\apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


movieId
1    4.000000
2    2.785714
3    2.500000
5    4.000000
6    3.928571
dtype: float64

Agora precisamos filtrar os filmes que o usuário já assistiu, pois eles não devem entrar nas recomendações.
Isso é feito com o filtro simples abaixo, que determina o índice dos filmes que o usuário possui rating observado.

In [48]:
moviesAlreadyWatched = userItemRatingMatrix.transpose()[user].dropna().index
moviesAlreadyWatched

Int64Index([ 111,  158,  173,  293,  596,  903, 1204, 1250, 1259, 1276, 1285,
            1358, 1639, 1687, 1747, 1876, 1909, 2001, 2019, 2072, 2174, 2502,
            2528, 2529, 2571, 2657, 2692, 2723, 2761, 2890, 3052, 3114, 3300,
            3751, 4641, 4975, 5952, 7090, 7153, 7361, 8368, 8636, 8784, 8874],
           dtype='int64', name='movieId')

In [49]:
#É calculada a média dos filmes sem rating observado.
avgRating = avgRating[~avgRating.index.isin(moviesAlreadyWatched)]

In [50]:
#Então é feito um teste com os 3 filmes com melhor recomendações, baseado na ordenação
#das médias de ratings dos vizinhos mais próximos.
N=3
topNMOVIEIDs = avgRating.sort_values(ascending=False).index[:N]

In [51]:
pd.Series(topNMOVIEIDs).apply(movieMeta)

0                                (Wilde (1997), Drama)
1    (Amazing Panda Adventure, The (1995), Adventur...
2    (Postman, The (1997), Action|Adventure|Drama|S...
Name: movieId, dtype: object

Tendo tudo isso, podemos encapsular em um método que executa todas essas etapas e retorna as recomendações, chamado de topN.

In [52]:
def topN(user,N=3):
    KnearestUsers = nearestNeighbors(user)
    NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
    avgRating = NNRatings.apply(np.nanmean).dropna()
    moviesAlreadyWatched = userItemRatingMatrix.transpose()[user].dropna().index
    avgRating = avgRating[~avgRating.index.isin(moviesAlreadyWatched)]
    ratingPredictedValue = avgRating.sort_values(ascending=False)
    #print(ratingPredictedValue.head(N))
    topNMOVIEIDs = avgRating.sort_values(ascending=False).index[:N]
    recommendation = pd.DataFrame(topNMOVIEIDs) 
    recommendation["title"] = recommendation["movieId"].apply(movieMeta)
    recommendation["Prediction"] = ratingPredictedValue.values[:N]
    return recommendation

In [53]:
faveMovies(3,10)

,userId,movieId,rating,timestamp,title
133,3,3949,5.0,1298863174,"(Sandlot, The (1993), Children|Comedy|Drama)"
113,3,1197,5.0,1298932770,"(Crash (1996), Drama|Thriller)"
101,3,318,5.0,1298862121,"(Crow, The (1994), Action|Crime|Fantasy|Thriller)"
103,3,356,5.0,1298862167,"(Mirage (1995), Action|Thriller)"
130,3,2959,5.0,1298862874,"(Bird on a Wire (1990), Action|Comedy|Romance)"
100,3,296,4.5,1298862418,"(Tom & Viv (1994), Drama)"
119,3,1721,4.5,1298923236,"(Return to Paradise (1998), Crime|Drama|Romanc..."
97,3,110,4.0,1298922049,"(Boomerang (1992), Comedy|Romance)"
129,3,2858,4.0,1298921825,"(Hidden, The (1987), Action|Horror|Sci-Fi)"
111,3,778,4.0,1298863157,"(Angel on My Shoulder (1946), Crime|Drama)"





Veja o exemplo abaixo, onde são recomendados 70 filmes, com base na média das distâncias, que funcionam como a similaridade, e faça o mesmo nas próximas células vazias para o usuário 8 e o usuário 9.

Note que temos uma coluna com o ID do filme, outra com seu título e outra com o rating predito para ele.

In [54]:
topN(6,70)

C:\Anaconda3\lib\site-packages\pandas\core\apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


,movieId,title,Prediction
0,1946,"(Wilde (1997), Drama)",5.000000
1,125,"(Amazing Panda Adventure, The (1995), Adventur...",5.000000
2,1361,"(Postman, The (1997), Action|Adventure|Drama|S...",5.000000
3,3260,"(Invisible Circus, The (2001), Drama|Romance)",5.000000
4,994,"(Terminator, The (1984), Action|Sci-Fi|Thriller)",5.000000
5,1264,"(Air Bud (1997), Children|Comedy)",5.000000
6,1260,"(George of the Jungle (1997), Children|Comedy)",5.000000
7,282,"(Stargate (1994), Action|Adventure|Sci-Fi)",5.000000
8,262,"(Outbreak (1995), Action|Drama|Sci-Fi|Thriller)",5.000000
9,2941,"(Light Years (Gandahar) (1988), Adventure|Anim...",5.000000


In [54]:
#Gere recomendações para o usuário 8. Um total de 80 recomendações.

In [55]:
topN(8,80)

C:\Anaconda3\lib\site-packages\pandas\core\apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


,movieId,title,Prediction
0,1947,"(Outside Ozona (1998), Comedy|Drama|Thriller)",5.000000
1,3071,"(Pumpkinhead (1988), Horror)",5.000000
2,1927,"(Heartburn (1986), Comedy|Drama)",5.000000
3,3007,"(Fun and Fancy Free (1947), Animation|Children...",5.000000
4,1354,(Midnight in the Garden of Good and Evil (1997...,5.000000
5,2730,"(Do the Right Thing (1989), Drama)",5.000000
6,6242,"(Anderson Tapes, The (1971), Crime|Drama|Thril...",5.000000
7,2132,"(It Came from Outer Space (1953), Sci-Fi)",5.000000
8,3089,"(Cell, The (2000), Drama|Horror|Thriller)",5.000000
9,913,"(Hustler White (1996), Romance)",5.000000


In [56]:
#Gere recomendações para o usuário 9. Um total de 60 recomendações.

In [56]:
topN(9,60)

C:\Anaconda3\lib\site-packages\pandas\core\apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


,movieId,title,Prediction
0,92,"(City Hall (1996), Drama|Thriller)",5.000000
1,1927,"(Heartburn (1986), Comedy|Drama)",5.000000
2,7502,"(Mother (Madeo) (2009), Crime|Drama|Mystery|Th...",5.000000
3,229,"(Just Cause (1995), Mystery|Thriller)",5.000000
4,1274,"(Leave It to Beaver (1997), Comedy)",5.000000
5,2726,"(...And Justice for All (1979), Drama|Thriller)",5.000000
6,7132,"(Blackadder Back & Forth (1999), Comedy)",5.000000
7,724,"(Vertigo (1958), Drama|Mystery|Romance|Thriller)",5.000000
8,1950,"(Hi-Lo Country, The (1998), Drama|Romance|West...",5.000000
9,6662,"(Dead Silence (2007), Horror|Mystery|Thriller)",5.000000


In [57]:
topN(1,60)

C:\Anaconda3\lib\site-packages\pandas\core\apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


,movieId,title,Prediction
0,549,"(Last Supper, The (1995), Drama|Thriller)",5.000000
1,92,"(City Hall (1996), Drama|Thriller)",5.000000
2,1354,(Midnight in the Garden of Good and Evil (1997...,5.000000
3,1031,"(Cyrano de Bergerac (1990), Comedy|Drama|Romance)",5.000000
4,1305,"(Full Monty, The (1997), Comedy|Drama)",5.000000
5,151,"(Johnny Mnemonic (1995), Action|Sci-Fi|Thriller)",5.000000
6,156,"(Lord of Illusions (1995), Horror)",5.000000
7,1125,"(Jaws (1975), Action|Horror)",5.000000
8,8228,"(Impossible, The (Imposible, Lo) (2012), Drama...",5.000000
9,188,"(Before the Rain (Pred dozhdot) (1994), Drama|...",5.000000


In [58]:
topN(2,60)

C:\Anaconda3\lib\site-packages\pandas\core\apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


,movieId,title,Prediction
0,5060,"(People Will Talk (1951), Comedy|Romance)",5.000000
1,923,"(Ruling Class, The (1972), Comedy|Drama)",5.000000
2,951,"(One Flew Over the Cuckoo's Nest (1975), Drama)",5.000000
3,968,"(Third Man, The (1949), Film-Noir|Mystery|Thri...",5.000000
4,969,"(Goodfellas (1990), Crime|Drama)",5.000000
5,1060,"(When Harry Met Sally... (1989), Comedy|Romance)",5.000000
6,1178,"(Pest, The (1997), Comedy)",5.000000
7,2797,"(Solaris (Solyaris) (1972), Drama|Mystery|Sci-Fi)",5.000000
8,1207,"(Inventing the Abbotts (1997), Drama|Romance)",5.000000
9,1211,"(Kissed (1996), Drama|Romance)",5.000000
